# Derivation of matrix elements

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as w
import diskwarp as dw
from IPython.display import clear_output, display

Define some symbols

In [ ]:
i_warp, phi_twist, PA_twist, i_obs, PA_obs = sp.symbols('i_warp, \phi_twist, PA_twist, i_obs, PA_obs', real=True)
x, y, z = sp.symbols('x, y, z', real=True)

Define the rotation matrices.

In [ ]:
def Rx(angle):
    return sp.Matrix([
    [1,       0,              0],
    [0,       sp.cos(angle), -sp.sin(angle)],
    [0,       sp.sin(angle),  sp.cos(angle)],
])

def Ry(angle):
    return sp.Matrix([
    [ sp.cos(angle), 0, sp.sin(angle)],
    [0,              1, 0],
    [-sp.sin(angle), 0, sp.cos(angle)],
])

def Rz(angle):
    return sp.Matrix([
    [sp.cos(angle), -sp.sin(angle), 0],
    [sp.sin(angle),  sp.cos(angle), 0],
    [0,              0,             1],
])

Define the initial coordinates $\vec p_0 = \begin{pmatrix}
x\\
y\\
z
\end{pmatrix}$

In [ ]:
p0 = sp.Matrix([x, y, z])

We apply the tilting and twisting to the disk coordinates which is a rotation first around $\vec x$ and then around $\vec z$, then the camera orientation

$R = R_z(PA_{obs}) \cdot R_x(i_{obs}) \cdot R_z(PA_{twist}) \cdot R_z(\phi_{twist}) \cdot R_x(i_{warp})$

In [ ]:
R_big = Rz(PA_obs) * Rx(i_obs) * Rz(PA_twist) * Rz(phi_twist) * Rx(i_warp)

In [ ]:
p1 =  R_big * p0

The result is rather messy ...

In [ ]:
sp.simplify(p1)

... so we rather let `sympy` write the fortran code for us. This is what is put into the `fortran.f90` file.

In [ ]:
from sympy.printing import fcode
code = [s + 'w = ' + fcode(p, standard=2003, source_format='free').replace('*', ' * ').replace('numpy', 'np').replace('\\', '') for s, p in zip('xyz', p1)]
print('\n\n'.join(code))

Or we generate numpy code, which is what we use in this notebook

In [ ]:
from sympy.printing.numpy import NumPyPrinter
pr = NumPyPrinter()
code = [s + '2 = ' + pr.doprint(p).replace('*', ' * ').replace('numpy', 'np').replace('\\', '') for s, p in zip('xyz', p1)]

print('\n\n'.join(code))

In [ ]:
def warp_points(x, y, z, i_warp = 0.0, phi_twist = 0.0, PA_twist = 0.0, i_obs = 0.0, PA_obs = 0.0):
    x2 = x * ((-np.sin(PA_obs) * np.sin(PA_twist) * np.cos(i_obs) + np.cos(PA_obs) * np.cos(PA_twist)) * np.cos(phi_twist) + (-np.sin(PA_obs) * np.cos(PA_twist) * np.cos(i_obs) - np.sin(PA_twist) * np.cos(PA_obs)) * np.sin(phi_twist)) + y * ((-(-np.sin(PA_obs) * np.sin(PA_twist) * np.cos(i_obs) + np.cos(PA_obs) * np.cos(PA_twist)) * np.sin(phi_twist) + (-np.sin(PA_obs) * np.cos(PA_twist) * np.cos(i_obs) - np.sin(PA_twist) * np.cos(PA_obs)) * np.cos(phi_twist)) * np.cos(i_warp) + np.sin(PA_obs) * np.sin(i_obs) * np.sin(i_warp)) + z * (-(-(-np.sin(PA_obs) * np.sin(PA_twist) * np.cos(i_obs) + np.cos(PA_obs) * np.cos(PA_twist)) * np.sin(phi_twist) + (-np.sin(PA_obs) * np.cos(PA_twist) * np.cos(i_obs) - np.sin(PA_twist) * np.cos(PA_obs)) * np.cos(phi_twist)) * np.sin(i_warp) + np.sin(PA_obs) * np.sin(i_obs) * np.cos(i_warp))
    y2 = x * ((-np.sin(PA_obs) * np.sin(PA_twist) + np.cos(PA_obs) * np.cos(PA_twist) * np.cos(i_obs)) * np.sin(phi_twist) + (np.sin(PA_obs) * np.cos(PA_twist) + np.sin(PA_twist) * np.cos(PA_obs) * np.cos(i_obs)) * np.cos(phi_twist)) + y * (((-np.sin(PA_obs) * np.sin(PA_twist) + np.cos(PA_obs) * np.cos(PA_twist) * np.cos(i_obs)) * np.cos(phi_twist) - (np.sin(PA_obs) * np.cos(PA_twist) + np.sin(PA_twist) * np.cos(PA_obs) * np.cos(i_obs)) * np.sin(phi_twist)) * np.cos(i_warp) - np.sin(i_obs) * np.sin(i_warp) * np.cos(PA_obs)) + z * (-((-np.sin(PA_obs) * np.sin(PA_twist) + np.cos(PA_obs) * np.cos(PA_twist) * np.cos(i_obs)) * np.cos(phi_twist) - (np.sin(PA_obs) * np.cos(PA_twist) + np.sin(PA_twist) * np.cos(PA_obs) * np.cos(i_obs)) * np.sin(phi_twist)) * np.sin(i_warp) - np.sin(i_obs) * np.cos(PA_obs) * np.cos(i_warp))
    z2 = x * (np.sin(PA_twist) * np.sin(i_obs) * np.cos(phi_twist) + np.sin(phi_twist) * np.sin(i_obs) * np.cos(PA_twist)) + y * ((-np.sin(PA_twist) * np.sin(phi_twist) * np.sin(i_obs) + np.sin(i_obs) * np.cos(PA_twist) * np.cos(phi_twist)) * np.cos(i_warp) + np.sin(i_warp) * np.cos(i_obs)) + z * (-(-np.sin(PA_twist) * np.sin(phi_twist) * np.sin(i_obs) + np.sin(i_obs) * np.cos(PA_twist) * np.cos(phi_twist)) * np.sin(i_warp) + np.cos(i_obs) * np.cos(i_warp))
    return x2, y2, z2

# Visualize Warped Disk

In [ ]:
# r and phi grid
r = np.linspace(0.5, 1.5, 20)
phi = np.linspace(0, 2 * np.pi, 30)

# compute cartesian values
x = r[:, None] * np.cos(phi)
y = r[:, None] * np.sin(phi)
z = np.zeros_like(x)

# compute warped positions
i_warp = dw.helper.warp(r, i_in=0, r0=1, dr=0.5)
phi_twist = dw.helper.twist(r, r0=1, dr=0.5)
xw, yw, zw = warp_points(x, y, z, i_warp=i_warp[:, None])

In [ ]:
%matplotlib widget

In [ ]:
## Controls

In [ ]:
#### Define Slider

slider_warp    = w.FloatSlider(orientation='horizontal', description='Warp:',         value=0.0, min=0.0, max=90.0)
slider_twist   = w.FloatSlider(orientation='horizontal', description='Twist:',        value=0.0, min=0.0, max=180.0)
slider_PAtwist = w.FloatSlider(orientation='horizontal', description='PA$_{twist}$:', value=0.0, min=0.0, max=360.0)
slider_i_obs   = w.FloatSlider(orientation='horizontal', description='i$_{obs}$:',    value=0.0, min=0.0, max=180.0)
slider_PA_obs  = w.FloatSlider(orientation='horizontal', description='PA$_{obs}$:',   value=0.0, min=0.0, max=360.0)

ui = [
    slider_warp,
    slider_twist,   
    slider_PAtwist, 
    slider_i_obs,
    slider_PA_obs
]

#slider.layout.margin = '0px 30% 0px 30%'
#slider.layout.width = '40%'

ui_box = w.VBox(ui)

In [ ]:
## Figure

In [ ]:
plt.ioff();

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_zlim(-1.5, 1.5)

lines_phi = []
for ir in range(x.shape[0]):
    lines_phi += plt.plot(xw[ir, :], yw[ir, :], zw[ir, :], lw=1, c='k')
    
lines_r = []
for iphi in range(x.shape[1]):
    lines_r += plt.plot(xw[:, iphi], yw[:, iphi], zw[:, iphi], lw=1, c='k')
    
#### Define Update Function

def update(change):
    
    # compute warped positions
    i_warp = dw.helper.warp(r, i_in=slider_warp.value,     r0=1, dr=0.5)
    phi_twist = dw.helper.twist(r, phi=slider_twist.value, r0=1, dr=0.5)
    
    xw, yw, zw = warp_points(
        x, y, z,
        i_warp=i_warp[:, None],
        phi_twist=phi_twist[:, None],
        PA_twist=np.deg2rad(slider_PAtwist.value),
        i_obs=np.deg2rad(slider_i_obs.value),
        PA_obs=np.deg2rad(slider_PA_obs.value)
    )
    
    for ir in range(x.shape[0]):
        lines_phi[ir].set_data(xw[ir, :], yw[ir, :])
        lines_phi[ir].set_3d_properties(zw[ir, :])

    for iphi in range(x.shape[1]):
        lines_r[iphi].set_data(xw[:, iphi], yw[:, iphi])
        lines_r[iphi].set_3d_properties(zw[:, iphi])
    
    fig.canvas.draw()
    fig.canvas.flush_events()

#### Setup output
    
for slider in ui:
    slider.observe(update, names='value')

In [ ]:
w.AppLayout(header=w.HTML(value="<h1><Warped Disk/h1>"),
          left_sidebar=ui_box,
          center=fig.canvas,
          right_sidebar=None,
          footer=None)

In [ ]:
#!voila --TagRemovePreprocessor.remove_cell_tags='{"hide"}' matrix_derivation.ipynb